In [1]:
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [2]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using plaidml.keras.backend backend.


In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [4]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

C:\Users\Ben\Anaconda3\envs\cvdl\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [5]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32, (3,3), input_shape=(32,32,3), activation='relu'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(64, (3,3)))
classifier.add(BatchNormalization())

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(100, activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10, activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

classifier.summary()

INFO:plaidml:Opening device "opencl_amd_gfx902.0"


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 30, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
batch_normalization_2 (Batch (None, 13, 13, 64)        256       
_________________________________________________________________
flatten_1 (Flatten)          (None, 10816)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               1081700   
__________

C:\Users\Ben\Anaconda3\envs\cvdl\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


In [6]:
classifier.fit(x_train, y_train, batch_size=100, epochs=30)

Epoch 1/30
50000/50000 [==============================] - 69s 1ms/step - loss: 1.4810 - acc: 0.5249
Epoch 2/30
50000/50000 [==============================] - 60s 1ms/step - loss: 0.9290 - acc: 0.6787
Epoch 3/30
50000/50000 [==============================] - 60s 1ms/step - loss: 0.7419 - acc: 0.7411
Epoch 4/30
50000/50000 [==============================] - 60s 1ms/step - loss: 0.6086 - acc: 0.7848
Epoch 5/30
50000/50000 [==============================] - 60s 1ms/step - loss: 0.4815 - acc: 0.8315
Epoch 6/30
50000/50000 [==============================] - 60s 1ms/step - loss: 0.3861 - acc: 0.8664
Epoch 7/30
50000/50000 [==============================] - 59s 1ms/step - loss: 0.3044 - acc: 0.8922
Epoch 8/30
50000/50000 [==============================] - 61s 1ms/step - loss: 0.2480 - acc: 0.9142
Epoch 9/30
50000/50000 [==============================] - 60s 1ms/step - loss: 0.1974 - acc: 0.9306
Epoch 10/30
50000/50000 [==============================] - 61s 1ms/step - loss: 0.1842 - acc: 0.9357

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [7]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[4.3025593e-05, 7.2822091e-05, 7.4027653e-04, 6.4473879e-01,
        2.7154344e-03, 3.5141101e-01, 1.5929305e-04, 1.1837249e-04,
        3.2105353e-07, 6.8845560e-07]], dtype=float32)